##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multi-Armed Bandits 소개

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/intro_bandit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/agents/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기

Multi-Armed Bandit(MAB)는 장기적으로 누적 보상을 극대화하기 위해 에이전트가 액션(무기)을 선택해야 하는 머신러닝 프레임워크입니다. 각 라운드에서 에이전트는 현재 상태(컨텍스트)에 대한 정보를 받은 다음 이 정보와 이전 라운드에서 수집한 경험을 기반으로 행동을 선택합니다. 각 라운드가 끝날 때 에이전트는 선택한 작업과 관련된 보상을 받습니다.

아마도 가장 순수한 예는 MAB에 그 이름을 빌려준 문제일 것입니다. `k` 슬롯 머신(one-armed bandits)에 직면했다고 가정하고, 어떤 것이 가장 좋은 지불금을 가지고 있지만 너무 많은 돈을 잃지 않는지 알아내야 합니다.

![Multi-Armed Bandits](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Las_Vegas_slot_machines.jpg/320px-Las_Vegas_slot_machines.jpg)

각 머신을 한 번 시도한 다음 가장 많이 지불한 머신을 선택하는 것은 좋은 전략이 아닙니다. 에이전트는 처음에는 운이 좋았지만 일반적으로 차선책인 머신을 선택하게 될 수 있습니다. 대신 에이전트는 더 많은 정보를 수집하기 위해 좋지 않은 머신을 선택하는 단계로 반복적으로 돌아와야 합니다. 이것이 Multi-Armed Bandits의 주요 과제입니다. 에이전트는 최적의 행동을 간과하지 않도록 사전 지식을 활용하는 것과 탐색 사이의 적절한 혼합을 찾아야 합니다.

MAB의 보다 실용적인 사례에는 학습자가 결정을 내릴 때마다 부수적인 정보가 포함됩니다. 이 부가 정보를 "컨텍스트"또는 "관찰"이라고 합니다.


## Multi-Armed Bandits 및 강화 학습

TF-Agents 라이브러리에 MAB Suite가있는 이유는 무엇입니까? RL과 MAB의 연관성은 무엇입니까? Multi-Armed Bandits는 강화 학습의 특별한 경우로 생각할 수 있습니다. [RL 소개](https://www.tensorflow.org/agents/tutorials/0_intro_rl) 를 인용하려면 :

*각 타임스텝에서 에이전트는 정책 $\pi(a_t|s_t)$에 따라 환경에 대한 행동을 취합니다. 여기서 $s_t$는 환경의 현재 관찰이며, 환경에서 보상 $r_{t+1}$과 다음 관측값 $s_{t+1}$을 받습니다. 목표는 보상(이익)의 합계를 극대화하기 위해 정책을 개선하는 것입니다.*

일반적인 RL 사례에서 다음 관찰 $ s_ {t + 1} $은 이전 상태 $ s_t $ 및 정책에서 취한 조치 $ a_t $에 따라 다릅니다. 이 마지막 부분은 MAB와 RL을 구분합니다. MAB에서 관찰 인 다음 상태는 에이전트가 선택한 동작에 의존하지 않습니다.

이러한 유사성을 통해 TF-Agent에 존재하는 모든 개념을 재사용 할 수 있습니다.

- **환경은** 관찰을 출력하고 보상으로 행동에 반응합니다.
- **정책** 은 관찰을 기반으로 조치를 출력하고
- **에이전트**는 이전 관찰-행동-보상 튜플을 기반으로 정책을 반복적으로 업데이트합니다.


## 버섯 환경

설명을 위해 "버섯 환경"이라는 장난감 예제를 사용합니다. 버섯 데이터세트([Schlimmer, 1981](https://archive.ics.uci.edu/ml/datasets/Mushroom))는 식용 및 독성 버섯의 레이블이 지정된 예로 구성됩니다. 특성에는 모양, 색상, 버섯의 여러 부분의 크기, 냄새 등이 포함됩니다.

![mushroom](https://archive.ics.uci.edu/ml/assets/MLimages/Large73.jpg)

모든 지도 학습 데이터세트와 마찬가지로, 버섯 데이터세트는 상황별 MAB 문제로 전환될 수 있습니다. [Riquelme 등. (2018)](https://arxiv.org/pdf/1802.09127.pdf)에서도 사용되는 방법을 사용합니다. 이 변환에서 에이전트는 버섯의 특징을 받아 먹거나 먹지 않기로 결정합니다. 식용 버섯을 먹으면 +5의 보상이 생기고, 독버섯을 먹으면 같은 확률로 +5 또는 -35가 됩니다. 버섯을 먹지 않으면 버섯의 종류와 관계없이 보상이 0이됩니다. 다음 표에는 보상 할당이 요약되어 있습니다.

> ```
>
> ```

```
       | edible | poisonous
```

----------- | -------- | ---------- 먹기 | +5 | -35 / +5 먹지 않기| 0 | 0

```

```

## LinUCB 에이전트

상황 별 적기 환경에서 잘 수행하려면 관찰을 고려할 때 각 행동의 보상 함수에 대한 좋은 추정이 필요합니다. 한 가지 가능성은 선형 함수로 보상 함수를 추정하는 것입니다. 즉, 모든 작업 $ i $에 대해 추정값이있는 $ \ theta_i \ in \ mathbb R ^ d $ 매개 변수를 찾으려고합니다.

$r_{t, i} \sim \langle v_t, \theta_i\rangle$

가능한 한 현실에 가깝습니다. 여기서 $ v_t \ in \ mathbb R ^ d $는 $ t $ 시간 단계에서받은 컨텍스트입니다. 그런 다음 에이전트가 예상치에 매우 확신이 있으면 $ \ arg \ max_ {1, ..., K} \ langle v_t, \ theta_k \ rangle $을 선택하여 예상되는 가장 높은 보상을 얻을 수 있습니다.

위에서 설명했듯이, 단순히 가장 좋은 예상 보상을 가진 손잡이(arm)을 선택하는 것은 좋은 전략으로 이어지지 않습니다. 선형 추정 에이전트에서 착취와 탐색을 혼합하는 방법에는 여러 가지가 있으며 가장 유명한 방법 중 하나는 선형 상한 신뢰 경계 (LinUCB) 알고리즘입니다 (예 : [Li et al. 2010 참조](https://arxiv.org/abs/1003.0146) ). LinUCB에는 두 가지 주요 구성 요소가 있습니다 (일부 세부 정보 생략).

1. Linear Least Squares: $\hat\theta_i\sim X^+_i r_i$를 사용하여 모든 손잡이(arm)의 매개변수에 대한 추정치를 유지합니다. $X_i$ 및 $r_i$는 팔 $i$이 선택된 라운드의 누적 컨텍스트 및 보상이고, $()^+$는 의사 역입니다.
2. 위 추정치에 대해 역공 분산 $ X_i ^ \ top X_i $로 정의 된 *신뢰 타원체를* 유지합니다.

LinUCB의 기본 아이디어는 "불확실성에 직면한 낙관주의"입니다. 에이전트는 해당 추정치의 분산에 해당하는 양만큼 추정치를 부스트하여 탐색을 통합합니다. 여기서 신뢰 타원체가 등장합니다. 모든 손잡이(arm)에 대한 낙관적 추정치는 $\hat r_i = \max_{\theta\in E_i}\langle v_t, \theta\rangle$이며, 여기서 $E_i$는 $\hat\theta_i$를 둘러싼 타원체입니다. 에이전트는 가장 좋아보이는 손잡이(arm)인 $\arg\max_i\hat r_i$를 선택합니다.

물론 위의 설명은 LinUCB가 수행하는 작업에 대한 직관적이지만 피상적인 요약입니다. 구현은 [여기](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/lin_ucb_agent.py)의 코드베이스에서 찾을 수 있습니다.

## 무엇 향후 계획?

Bandits 라이브러리에 대한 자세한 튜토리얼을 보려면 [Bandits 튜토리얼](https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/bandits_tutorial.ipynb)을 살펴보세요. 대신 바로 우리의 라이브러리 탐색을 시작하기 원하는 경우 [여기](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits)에서 찾을 수 있습니다. 교육을 바로 시작하고 싶다면 [여기](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2/train_eval_mushroom.py)에서 위에서 설명한 LinUCB를 사용하는 버섯 환경과 [여기](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2)에서 엔드 투 엔드 예시를 살펴보세요. 